# การค้นหาเชิงลำดับชั้น (Hierarchical Search)

การค้นหาเชิงลำดับชั้นเป็นอัลกอริทึมการวางแผนที่ทำงานในระดับนามธรรมที่สูงกว่า <br>
แทนที่จะใช้ “การกระทำ” แบบการวางแผนคลาสสิก (บทที่ 10) ซึ่งเป็นการกระทำปฐมภูมิ (primitive actions) ที่แยกย่อยไม่ได้ เราจะใช้ “การกระทำระดับสูง” หรือ HLA (High-Level Actions) (ดูเพิ่มใน planning.ipynb) <br>

## การทำให้ละเอียดยิ่งขึ้น (Refinements)

แต่ละ __HLA__ จะมีการทำให้ละเอียดยิ่งขึ้นหนึ่งแบบหรือมากกว่า ให้กลายเป็นลำดับของการกระทำ ซึ่งแต่ละรายการอาจเป็น HLA อื่น หรือเป็นการกระทำปฐมภูมิ (ที่โดยนิยามแล้วจะไม่มีการทำให้ละเอียดยิ่งขึ้น) <br>
ตัวอย่าง:
-  (a) การกระทำระดับสูง “ไปสนามบินซานฟรานซิสโก” (Go(Home, SFO)) อาจมีการทำให้ละเอียดยิ่งขึ้นได้สองแบบ คือ “ขับรถไปสนามบินซานฟรานซิสโก” และ “นั่งแท็กซี่ไปสนามบินซานฟรานซิสโก”
<br>
-  (b) ตัวอย่างการทำให้ละเอียดยิ่งขึ้นแบบเวียนเกิดซ้ำ (recursive) ในโลกเครื่องดูดฝุ่น: เพื่อไปยังจุดหมาย ให้ก้าวไปหนึ่งก้าว แล้วจึงไปยังจุดหมายนั้น
<br>
![title](images/refinement.png)
<br>
-  __implementation__: การทำให้ HLA ละเอียดยิ่งขึ้นที่มีแต่การกระทำปฐมภูมิเท่านั้น เรียกว่า implementation ของ HLA
-  การทำให้แผนระดับสูง (ลำดับของ HLA) เป็น implementation คือการนำ implementation ของแต่ละ HLA ในลำดับนั้นมาต่อกัน
- แผนระดับสูงจะถือว่า __บรรลุเป้าหมาย__ จากสถานะที่กำหนด หากมีอย่างน้อยหนึ่ง implementation ของมันที่บรรลุเป้าหมายจากสถานะนั้น
<br>

พารามิเตอร์ของฟังก์ชัน refinements คือ:
-  __hla__: HLA ที่เราต้องการคำนวณชุดการทำให้ละเอียดยิ่งขึ้น
- __state__: ฐานความรู้ของปัญหาปัจจุบัน (Problem.init)
- __library__: ลำดับชั้นของการกระทำในปัญหาการวางแผน



In [1]:
from planning import * 
from notebook import psource

In [11]:
psource(Problem.refinements)

## การค้นหาเชิงลำดับชั้น (Hierarchical search)

การค้นหาเชิงลำดับชั้นเป็นอัลกอริทึมค้นหาแบบกว้างก่อน (breadth-first) สำหรับการวางแผนแบบเดินหน้า (forward planning) ในพื้นที่ของการทำให้ละเอียดยิ่งขึ้นของ HLA (คือ ทำซ้ำโดยเลือก HLA ในแผนปัจจุบัน แล้วแทนที่ด้วยหนึ่งในชุดการทำให้ละเอียดยิ่งขึ้นของมัน จนกว่าแผนจะบรรลุเป้าหมาย)

<br>
อินพุตของอัลกอริทึมคือ: problem และ hierarchy
-  __problem__: ออบเจ็กต์ชนิด Problem
-  __hierarchy__: ดิกชันนารีที่ระบุการกระทำทั้งหมดและลำดับการดำเนินการ
<br>

ในคำเรียกใช้ระดับบนสุด (top level call) ค่า initialPlan มี [act] (คือ การกระทำที่จะถูกดำเนินการ)


In [16]:
psource(Problem.hierarchical_search)

## ตัวอย่าง

สมมติว่ามีคนต้องการไปสนามบิน วิธีที่เป็นไปได้คือไปด้วยแท็กซี่หรือขับรถไปสนามบิน <br>
การกระทำทั้งสองมีเงื่อนไขก่อน (preconditions) และผลลัพธ์ (effects) บางอย่าง
ถ้าคุณไปด้วยแท็กซี่ คุณต้องมีเงินสด ส่วนถ้าคุณขับรถ คุณต้องมีรถยนต์ <br>
ดังนั้นเราจึงกำหนดลำดับชั้นของการกระทำที่เป็นไปได้ดังต่อไปนี้

##### ลำดับชั้น (hierarchy)


In [2]:
library = {
        'HLA': ['Go(Home,SFO)', 'Go(Home,SFO)', 'Drive(Home, SFOLongTermParking)', 'Shuttle(SFOLongTermParking, SFO)', 'Taxi(Home, SFO)'],
        'steps': [['Drive(Home, SFOLongTermParking)', 'Shuttle(SFOLongTermParking, SFO)'], ['Taxi(Home, SFO)'], [], [], []],
        'precond': [['At(Home) & Have(Car)'], ['At(Home)'], ['At(Home) & Have(Car)'], ['At(SFOLongTermParking)'], ['At(Home)']],
        'effect': [['At(SFO) & ~At(Home)'], ['At(SFO) & ~At(Home) & ~Have(Cash)'], ['At(SFOLongTermParking) & ~At(Home)'], ['At(SFO) & ~At(LongTermParking)'], ['At(SFO) & ~At(Home) & ~Have(Cash)']] }




การกระทำที่เป็นไปได้มีดังนี้:

In [3]:
go_SFO = HLA('Go(Home,SFO)', precond='At(Home)', effect='At(SFO) & ~At(Home)')
taxi_SFO = HLA('Taxi(Home,SFO)', precond='At(Home)', effect='At(SFO) & ~At(Home) & ~Have(Cash)')
drive_SFOLongTermParking = HLA('Drive(Home, SFOLongTermParking)', 'At(Home) & Have(Car)','At(SFOLongTermParking) & ~At(Home)' )
shuttle_SFO = HLA('Shuttle(SFOLongTermParking, SFO)', 'At(SFOLongTermParking)', 'At(SFO) & ~At(LongTermParking)')

สมมติว่า (เงื่อนไขเริ่มต้นของเรา) คือเราอยู่ที่บ้านและเรามีเงินสดและมีรถยนต์ และเป้าหมายของเราคือไปถึง SFO โดยยังคงมีเงินสดอยู่ และการกระทำที่เป็นไปได้คือชุดด้านบน <br>
##### ดังนั้นปัญหาของเราคือ: 

In [4]:
prob = Problem('At(Home) & Have(Cash) & Have(Car)', 'At(SFO) & Have(Cash)', [go_SFO])

##### การทำให้ละเอียดยิ่งขึ้น (Refinements)

การทำให้ละเอียดยิ่งขึ้นของการกระทำ Go(Home, SFO) ถูกกำหนดเป็น: <br>
['Drive(Home,SFOLongTermParking)', 'Shuttle(SFOLongTermParking, SFO)'], ['Taxi(Home, SFO)']

In [5]:
for sequence in Problem.refinements(go_SFO, prob, library):
    print (sequence)
    print([x.__dict__ for x in sequence ], '\n')

[HLA(Drive(Home, SFOLongTermParking)), HLA(Shuttle(SFOLongTermParking, SFO))]
[{'name': 'Drive', 'args': (Home, SFOLongTermParking), 'precond': [At(Home), Have(Car)], 'effect': [At(SFOLongTermParking), NotAt(Home)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}, {'name': 'Shuttle', 'args': (SFOLongTermParking, SFO), 'precond': [At(SFOLongTermParking)], 'effect': [At(SFO), NotAt(LongTermParking)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}] 

[HLA(Taxi(Home, SFO))]
[{'name': 'Taxi', 'args': (Home, SFO), 'precond': [At(Home)], 'effect': [At(SFO), NotAt(Home), NotHave(Cash)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}] 



รันการค้นหาเชิงลำดับชั้น
##### การเรียกใช้ระดับบนสุด (Top level call)

In [6]:
plan= Problem.hierarchical_search(prob, library)
print (plan, '\n')
print ([x.__dict__ for x in plan])

[HLA(Drive(Home, SFOLongTermParking)), HLA(Shuttle(SFOLongTermParking, SFO))] 

[{'name': 'Drive', 'args': (Home, SFOLongTermParking), 'precond': [At(Home), Have(Car)], 'effect': [At(SFOLongTermParking), NotAt(Home)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}, {'name': 'Shuttle', 'args': (SFOLongTermParking, SFO), 'precond': [At(SFOLongTermParking)], 'effect': [At(SFO), NotAt(LongTermParking)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}]


## ตัวอย่างที่ 2

In [7]:
library_2 = {
        'HLA': ['Go(Home,SFO)', 'Go(Home,SFO)', 'Bus(Home, MetroStop)', 'Metro(MetroStop, SFO)' , 'Metro(MetroStop, SFO)', 'Metro1(MetroStop, SFO)', 'Metro2(MetroStop, SFO)'  ,'Taxi(Home, SFO)'],
        'steps': [['Bus(Home, MetroStop)', 'Metro(MetroStop, SFO)'], ['Taxi(Home, SFO)'], [], ['Metro1(MetroStop, SFO)'], ['Metro2(MetroStop, SFO)'],[],[],[]],
        'precond': [['At(Home)'], ['At(Home)'], ['At(Home)'], ['At(MetroStop)'], ['At(MetroStop)'],['At(MetroStop)'], ['At(MetroStop)'] ,['At(Home) & Have(Cash)']],
        'effect': [['At(SFO) & ~At(Home)'], ['At(SFO) & ~At(Home) & ~Have(Cash)'], ['At(MetroStop) & ~At(Home)'], ['At(SFO) & ~At(MetroStop)'], ['At(SFO) & ~At(MetroStop)'], ['At(SFO) & ~At(MetroStop)'] , ['At(SFO) & ~At(MetroStop)'] ,['At(SFO) & ~At(Home) & ~Have(Cash)']] 
        }

In [8]:
plan_2 = Problem.hierarchical_search(prob, library_2)
print(plan_2, '\n')
print([x.__dict__ for x in plan_2])

[HLA(Bus(Home, MetroStop)), HLA(Metro1(MetroStop, SFO))] 

[{'name': 'Bus', 'args': (Home, MetroStop), 'precond': [At(Home)], 'effect': [At(MetroStop), NotAt(Home)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}, {'name': 'Metro1', 'args': (MetroStop, SFO), 'precond': [At(MetroStop)], 'effect': [At(SFO), NotAt(MetroStop)], 'duration': 0, 'consumes': {}, 'uses': {}, 'completed': False}]
